In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]

def get_losers():
    
    print('Losers'.center(45))
    print()
    print('Symbol'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Loss')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers("losers"))):
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc)+'%')
def get_gainers():
    print('Gainers'.center(45))
    print()
    print('Symbol'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Gain')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers())):
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc)+'%')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\benjamin.beale\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
if __name__ == '__main__':
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    print("{:<30}".format('–' * 45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    print("{:<30}".format('–' * 45))
    bears = selector.bearish_candlesticks(64, 20)

                   Gainers                   

Symbol     Last        Change      Pct Gain
–––––––––––––––––––––––––––––––––––––––––––––
AMHpF      $25.7095    $25.535     14607.952%
MHpC       $5.6001     $5.45       3633.4%
SGLBW      $0.7        $0.6        600%
TYp        $53.5       $41.94      362.803%
DRIOW      $0.0387     $0.026      214.634%
ALGRR      $0.47       $0.23       95.833%
CpN        $27.6       $12.35      80.984%
HJLIW      $0.25       $0.11       78.571%
ALGRW      $0.33       $0.13       65%
WRLSW      $0.32       $0.09       38.949%
OXBRW      $0.04       $0.01       33.333%
NDRAW      $0.370899   $0.09       32.087%
XPEL       $14.14      $3.33       30.805%
NSPR.WS    $0.0073     $0.002      28.07%
ZVZZT      $16.63      $3.64       28.022%
PHIO       $0.32       $0.07       27.949%
VERO       $6.3        $1.3        26%
CABA       $13.92      $2.8        25.18%
CARB       $22.95      $4.55       24.728%
ZWZZT      $15.1       $2.95       24.28%
AXAS       $

sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


Ticker: PINC		Poolsize: 20		Result Length: 10
Ticker: PGRE		Poolsize: 20		Result Length: 11


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


Ticker: MERC		Poolsize: 20		Result Length: 12
Ticker: ANET		Poolsize: 20		Result Length: 13
Ticker: GPX		Poolsize: 20		Result Length: 14


...

Ticker: LEG		Poolsize: 20		Result Length: 18


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


Ticker: CTS		Poolsize: 20		Result Length: 19


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


Ticker: DGX		Poolsize: 20		Result Length: 20


In [3]:
ticker = "BAX"

In [4]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

In [5]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

In [6]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format('–' * 45))
print("Stocktwits:\t{} score: {}\nTwitter:\t{} score:{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

Sentiment score for BAX
–––––––––––––––––––––––––––––––––––––––––––––
Stocktwits:	positive score: 0.9598
Twitter:	positive score:0.7177
Average:	0.83875


In [7]:
# alpaca_api.polygon.historic_trades(ticker, '2019-10-10', limit=10)
# alpaca_api.polygon.news(ticker)
# alpaca_api.polygon.last_trade(ticker)
# alpaca_api.polygon.last_quote(ticker)
# alpaca_api.polygon.company(ticker)
# alpaca_api.polygon.dividends(ticker)
# alpaca_api.polygon.financials(ticker)

In [8]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [9]:
stocktwitstext 

'$BAX Unusual Options Activity surfacing today. More than 10k options have traded, mainly calls. Why not learn how to do this yourself for Free? https://optionsgeek.com/get-started/the-ultimate-guide-to-unusual-options-activity/\nAn Altman-Z score of 5.01 indicates that $BAX is not in any danger for bankruptcy at the moment. https://www.chartmill.com/stock/analyzer/stock/BAX?view=fundamental-analysis&amp;key=87cb839a-7557-4b8b-a9da-a159e6f54ea9&amp;utm_source=stocktwits&amp;utm_medium=FA&amp;utm_content=HEALTH&amp;utm_campaign=social_tracking\nConservative investment portfolio: Baxter International($BAX), Sysco($SYY),… http://dlvr.it/RJ3GqS #correlations #stocks #stockratings\nCheck this portfolio in December 2019: Alcoa($AA), Baxter International($BAX),… https://www.macroaxis.com/invest/marketRiskAndReturn?s=AA,BAX,SLB,PWR,SJM,XRAY,UMBF,PH,SCOR,NRG,MAR,STBA #stocks #earnings\nBullish portfolio: CenturyLink($CTL), Baxter International($BAX), City… http://dlvr.it/RJ23yK #correlations #s

In [10]:
tweets 

"short % to float of $BAX is 0.09 at 2019-05-31 https://t.co/MO8SG3dwv5 $CAH 5.23% $FMS 0.09%\n\n$BAX in Uptrend: RSI indicator exits oversold zone. View odds for this and other indicators:… https://t.co/uuIRDt179D\n\n$BAX in Uptrend: RSI indicator exits oversold zone. View odds for this and other indicators:… https://t.co/raMQQ6h7PS\n\n$BAX in Uptrend: RSI indicator exits oversold zone. View odds for this and other indicators:… https://t.co/b1c5y2PbKF\n\n$CRY short ratio is 1.77 at 2019-05-31 https://t.co/MOTsPLZxEs $BSX 1.99 $JNJ 1.87 $SYK 2.59 $BAX 2.13\n\nTop #moneyflow for #sp500 today\n$AAPL $BAC $MU $MRK $T $BAX $XRAY $MS $CVS $BA ...\nhttps://t.co/S59zN0R1wP\n\nStock tip of the hour: everyone should be buy buy buying BAX!\n\n$BAX in Uptrend: RSI indicator exits oversold zone. View odds for this and other indicators:… https://t.co/GlJPJoCvZU\n\nA Quick Ratio of 2.10 indicates that $BAX has no problem at all paying its short term obligations.… https://t.co/zDI5V3szPQ\n\nTop stock

In [ ]:
tenquant_data